In [ ]:
import re
import json
import pickle
import pandas as pd

from sklearn.metrics import classification_report

In [ ]:
with open("/Utilisateurs/umushtaq/am_reasoning/saved_models/pe_pipeline_p3new_Meta-Llama-3.1-70B-Instruct-bnb-4bit/pe_pipeline_results_15.pickle", 'rb') as f:
    
    results = pickle.load(f)

In [93]:
grounds = results['grounds']
predictions = results['predictions']

In [94]:
grounds

['### Output:\n{"argument_types": [\'C\', \'M\', \'P\', \'P\', \'P\', \'C\', \'P\', \'P\', \'P\', \'C\', \'M\']}\n{"relation_types": [(3, 6, \'S\'), (4, 6, \'S\'), (5, 6, \'S\'), (7, 10, \'S\'), (8, 9, \'S\'), (9, 10, \'S\'), (1, 2, \'A\'), (1, 11, \'A\'), (6, 2, \'S\'), (6, 11, \'S\'), (10, 2, \'S\'), (10, 11, \'S\')]}\n',
 '### Output:\n{"argument_types": [\'M\', \'C\', \'P\', \'C\', \'P\', \'P\', \'P\', \'P\', \'P\', \'C\', \'C\', \'M\']}\n{"relation_types": [(3, 2, \'S\'), (5, 4, \'A\'), (6, 5, \'A\'), (7, 4, \'S\'), (8, 10, \'S\'), (9, 10, \'S\'), (2, 1, \'S\'), (2, 12, \'S\'), (4, 1, \'S\'), (4, 12, \'S\'), (10, 1, \'S\'), (10, 12, \'S\'), (11, 1, \'A\'), (11, 12, \'A\')]}\n',
 '### Output:\n{"argument_types": [\'M\', \'C\', \'C\', \'P\', \'P\', \'P\', \'P\', \'C\', \'P\', \'P\', \'C\', \'P\', \'P\', \'P\', \'P\', \'P\', \'P\', \'P\', \'M\']}\n{"relation_types": [(4, 3, \'S\'), (5, 6, \'S\'), (6, 3, \'S\'), (7, 6, \'S\'), (9, 8, \'S\'), (10, 8, \'S\'), (12, 11, \'S\'), (13, 11, \

### Process Grounds

In [95]:
def process_grounds(grounds):
    
    arg_types = []
    #rel_pairs = []
    rel_types = []
    
    for grounds in grounds:
        
        args = grounds.split("\n")[1].replace("'", '"')
        arg_types.append(json.loads(args)["argument_types"])
        
        rels = re.sub(r'\((\d+), (\d+)\)', r'[\1, \2]', grounds.split("\n")[2])
        rels = eval(rels)["relation_types"]
        #rels = [(tuple(pair), label) for pair, label in rels]
        rel_types.append(rels)
        
    return arg_types, rel_types

In [96]:
grounds_acc, grounds_aric  = process_grounds(grounds)

In [97]:
len(grounds_acc), len(grounds_aric)

(80, 80)

### Process Predictions

In [98]:
def process_predictions(predictions):
    
    arg_types = []
    rel_types = []
    
    for pred in predictions:
        
        args = pred.split("\n")[1].replace("'", '"')
        arg_types.append(json.loads(args)["argument_types"])
        
        rels = re.sub(r'\((\d+), (\d+)\)', r'[\1, \2]', pred.split("\n")[2])
        rels = eval(rels)["relation_types"]
        #rels = [(tuple(pair), label) for pair, label in rels]
        rel_types.append(rels)
        
    return arg_types, rel_types

In [99]:
predictions_acc, predictions_aric  = process_predictions(predictions)

In [100]:
len(predictions_acc), len(predictions_aric)

(80, 80)

### Compute ACC metrics

In [101]:
bad_idx = []

for idx, (g, p) in enumerate(zip(grounds_acc, predictions_acc)):
    
    if len(p) != len(g):
        print(idx)
        bad_idx.append(idx)
        #predictions_acc[idx] = p[:len(g)]

In [102]:
grounds_acc = [elem for idx, elem in enumerate(grounds_acc) if idx not in bad_idx]
predictions_acc = [elem for idx, elem in enumerate(predictions_acc) if idx not in bad_idx]

In [103]:
grounds = [elem for sublist in grounds_acc for elem in sublist]
predictions = [elem for sublist in predictions_acc for elem in sublist]

In [104]:
len(grounds), len(predictions)

(1266, 1266)

In [105]:
print(classification_report(grounds, predictions, digits=3))

              precision    recall  f1-score   support

           C      0.859     0.839     0.849       304
           M      0.962     0.980     0.971       153
           P      0.946     0.951     0.948       809

    accuracy                          0.927      1266
   macro avg      0.922     0.923     0.923      1266
weighted avg      0.927     0.927     0.927      1266



### Compute ARIC metrics

In [109]:
df = pd.read_csv("/Utilisateurs/umushtaq/am_reasoning/data_files/pe_dataset_final_c.csv", index_col=0)

In [110]:
df.columns

Index(['ann_file_name', 'argument_ids', 'argument_types', 'arg_start_bounds',
       'arg_end_bounds', 'argument_components', 'source_args', 'target_args',
       'relation_types', 'stance_types', 'essay_title', 'essay_text',
       'source_acs', 'target_acs', 'relation_pairs', 'stance_pairs', 'split',
       'sorted_acs', 'sorted_ac_types', 'para_acs', 'para_ac_types',
       'paras_structure', 'para_ac_all_pairs', 'para_ac_rels',
       'para_ac_stances'],
      dtype='object')

In [111]:
df = df[df.split == "TEST"].reset_index(drop=True)

In [113]:
def flatten(l):
    
    l = eval(l)
    #l = [eval(elem) for elem in l]
    return [elem for sl in l for elem in sl]

In [114]:
para_rels = []

for l in df.para_ac_rels:
    para_rels.append(flatten(l))

In [125]:
all_links = []

for i, j in zip(para_rels, df.para_ac_stances):
    j = eval(j)
    all_links.append(i + j)

In [126]:
len(all_links)

80

In [127]:
sum([len(elem) for elem in all_links])

5508

In [130]:
all_links = [
    [(a, b, 'S' if relation == 'supports' else relation) for (a, b, relation) in sublist]
    for sublist in all_links
]

In [132]:
all_links = [
    [(a, b, 'A' if relation == 'attacks' else relation) for (a, b, relation) in sublist]
    for sublist in all_links
]

In [134]:
def get_preds(all_links_e, predictions_e):
    
    preds_p = []
    
    # for pred in predictions_e:
        
    #     if pred in all_links_e:
            
    #         correct_predictions.append(pred[2])
            
    for link in all_links_e:
        
        if link in predictions_e:
            
            preds_p.append(link[2])
        else:
            preds_p.append("NR")
            
    return preds_p

In [ ]:
preds = []

for i, j in zip(all_links, predictions_aric):
    
    preds.append(get_preds(i, j))

In [136]:
len(preds)

80

In [137]:
preds = [elem for sl in preds for elem in sl]

In [138]:
len(preds)

5508

In [142]:
all_links = [elem[2] for sl in all_links for elem in sl]

In [144]:
len(all_links), len(preds)

(5508, 5508)

In [145]:
print(classification_report(all_links, preds, digits=3))

              precision    recall  f1-score   support

           A      1.000     0.512     0.677       129
          NR      0.913     1.000     0.954      4113
           S      1.000     0.739     0.850      1266

    accuracy                          0.928      5508
   macro avg      0.971     0.750     0.827      5508
weighted avg      0.935     0.928     0.924      5508

